In [1]:
import pandas as pd
import os
from itertools import chain
import datetime
import time
from tqdm import tqdm

In [2]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [3]:
Program_Start_Time  = time.time()
print("Current file location: "+ os.getcwd())
print("Login WBI : "+ os.getlogin())

Current file location: c:\Planning S&P\Project\SCB_PUI
Login WBI : nxf83451


### Read Input Reference (PUI and JDA)

In [4]:
print("File loading ....")

df_PUI_raw = pd.read_excel(r"./Input/Supply_Chain_PUI.xlsx", sheet_name="Raw")
df_JDA_input = pd.read_excel(r"./Input/Input_JDA_List.xlsx")

df_PUI_raw["EFF_FR_DATE"] = df_PUI_raw["EFF_FR_DATE"].astype(str).str.split(" ", expand=True)[0]
df_PUI_raw["EFF_TO_DATE"] = df_PUI_raw["EFF_TO_DATE"].astype(str).str.split(" ", expand=True)[0]

File loading ....


In [5]:
df_PUI_raw["COMP_12NC"] = df_PUI_raw["1_12NC_LIST"].copy()
df_PUI_raw["SingleComp_index"] = df_PUI_raw.index
df_PUI_raw.loc[(df_PUI_raw["COMP_12NC_LIST"].str.contains(",")) & (df_PUI_raw["TYPE"] == "ICAM"), "COMP_12NC"] = df_PUI_raw.loc[(df_PUI_raw["COMP_12NC_LIST"].str.contains(",")) & (df_PUI_raw["TYPE"] == "ICAM"), "COMP_12NC_LIST"]
df_PUI_raw["COMP_12NC"] = df_PUI_raw["COMP_12NC"].astype(str)

#Get component 12NC and extend them if they are mutlicomponent
MutiComp_stack = pd.DataFrame([])
temp = df_PUI_raw.loc[df_PUI_raw["COMP_12NC"].str.contains(",")]
temp.loc[temp["COMP_12NC"] .str.contains(","), "MutiComp"] = temp.loc[temp["COMP_12NC"].str.contains(","), "COMP_12NC"]


temp["SingleComp_index"] = pd.NaT
temp.loc[temp["COMP_12NC"].str.contains(","), "MutiComp_index"] = temp.loc[temp["COMP_12NC"].str.contains(",")].index
temp["MutiComp_index"] = temp["MutiComp_index"].astype(int)

def split_rows(df, column):
    # Create an empty DataFrame to store the new rows
    new_df = pd.DataFrame(columns=df.columns)

    for _, row in df.iterrows():
        # Check if the value in the specific column contains a comma
        if "," in str(row[column]):
            # Split the value by comma
            split_values = str(row[column]).split(',')
            for value in split_values:
                # Create a new row with the split value
                new_row = row.copy()
                new_row[column] = value.strip()  # Remove any leading/trailing whitespace
                new_df = new_df.append(new_row, ignore_index=True)
        else:
            # If no comma, just append the row as is
            new_df = new_df.append(row, ignore_index=True)
    
    return new_df

df_extend_multicomp = split_rows(temp, 'COMP_12NC')
df_extend_multicomp["COMP_12NC"] = df_extend_multicomp["COMP_12NC"].astype('str')

In [6]:
df_agg = pd.concat([df_PUI_raw, df_extend_multicomp])
df_agg.reset_index(drop = True, inplace =True)

print("Length of the original df: {}".format(len(df_PUI_raw)))
print("Length of the extended Multi-Component df: {}".format(len(df_extend_multicomp)))
print("Length of the Aggregated df: {}, {}".format(len(df_agg), len(df_agg) == (len(df_PUI_raw) + len(df_extend_multicomp))))

df_agg_final = df_agg[~df_agg.index.isin(df_agg["MutiComp_index"].dropna())]

print("Length of the Aggreaged df and drop multiple list: {}\n".format(len(df_agg_final)))


# Dict for adjacent 12NC
df_agg_final["PART_12NC"] = df_agg_final["PART_12NC"].astype("int64")
df_agg_final["COMP_12NC"] = df_agg_final["COMP_12NC"].astype("int64")
print(df_agg_final.info())


Length of the original df: 59929
Length of the extended Multi-Component df: 1048
Length of the Aggregated df: 60977, True
Length of the Aggreaged df and drop multiple list: 60462

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60462 entries, 0 to 60976
Data columns (total 33 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   HEADING           60462 non-null  object 
 1   ACTION            0 non-null      float64
 2   PPF               25225 non-null  object 
 3   TG5               56404 non-null  object 
 4   PART_12NC         60462 non-null  int64  
 5   TYPE              60462 non-null  object 
 6   LEGACY_PART       271 non-null    object 
 7   LOC               60462 non-null  object 
 8   PLANT             60462 non-null  object 
 9   OPTIONS           60462 non-null  object 
 10  STATUS            60462 non-null  object 
 11  EFF_FR_DATE       60462 non-null  object 
 12  EFF_TO_DATE       60462 non-null  object 
 13  C

In [7]:
# Dict for adjacent 12NC
prod = {}
for x in df_agg_final["PART_12NC"].unique():
    prod[x] = df_agg_final.loc[df_agg_final['PART_12NC'] == x, "COMP_12NC"].to_list()
print(len(prod))

# Dict for Matertial Type
material_12nc_type_list = [["PART_12NC", "TYPE"],
                           ["1_12NC_LIST", "1_CLASS"], 
                           ["2_12NC_LIST", "2_CLASS"], 
                           ["3_12NC_LIST", "3_CLASS"], 
                           ["4_12NC_LIST", "4_CLASS"]]


def Merge(dict1, dict2):
    res = {**dict1, **dict2}
    return res

Mtype = dict()
for material_12nc_type in (material_12nc_type_list):
    sub_material = df_agg[material_12nc_type[0]].dropna()
    sub_type = df_agg[material_12nc_type[1]].dropna()
    sub_Mtype = dict(zip(sub_material,sub_type))
    Mtype = Merge(Mtype, sub_Mtype)


49229


### PUI_Structure_Rolling Process

In [8]:
class PUI_Structure_Rolling:
    def __init__(self):
        self.SC_Num = 0
        self.PUI_anlyzied_slice_stack = pd.DataFrame([])
        self.df_all_valid_supply_path = pd.DataFrame([])
        self.PUI_stack = pd.DataFrame([])
        self.PUI_all_stack = pd.DataFrame([])

    def main(self, input_SC_dataframe, input_PUI_dataframe, input_PUI_ref_before_drop_multi_comp):
        self.input_SC_List_len = len(input_SC_dataframe)

        print("Rolling Supply Path ===========================================================")
        for prod_item, consumed_item, consumed_plant, BOM_id in tqdm(zip(input_SC_dataframe["ITEM_NAME"], input_SC_dataframe["CONSUMED_ITEM"], input_SC_dataframe["PLANT"], input_SC_dataframe["ITEM_BOM_RT_ID"]), total = len(input_SC_dataframe)):
            self.SC_Num += 1
            all_paths_list = self.find_unique_path(prod, prod_item, consumed_item) # Get all applicable paths (prod to comp) -> list
            if len(all_paths_list) == 0:
                continue
            self.df_all_valid_supply_path = pd.concat([self.df_all_valid_supply_path, self.Mtype_Aggregation(all_paths_list)]) # Get all applicable supply chain with Mtype Mapping -> dataframe
            df_SC_analyezd_Stack = self.SC_Structure_Extraction(input_PUI_dataframe, all_paths_list, consumed_plant, BOM_id) # Extract each supply path and check if the supply path is valid
            
            #Result concating for each input supply path combination
            df_SC_analyezd_Stack["SC Index"] = self.SC_Num
            self.PUI_anlyzied_slice_stack = pd.concat([self.PUI_anlyzied_slice_stack, df_SC_analyezd_Stack]) 

        print("\nMulti-Component Checking ======================================================")
        df_component_analyzed_result  = self.MultiComponent_Checking(self.PUI_anlyzied_slice_stack, input_SC_dataframe)
        if len(df_component_analyzed_result) != 0:
            df_component_analyzed_result.insert(0, 'SC Index', df_component_analyzed_result.pop('SC Index'))
            df_component_analyzed_result.insert(2, 'BOM_ID', df_component_analyzed_result.pop('BOM_ID'))

        print("\nPUI Mapping Result ============================================================")
        df_PUI_mapping = self.PUI_result_mapping(input_PUI_ref_before_drop_multi_comp, df_component_analyzed_result)
        
        #Result proprocessing
        if len(df_PUI_mapping) != 0:
            df_PUI_mapping["PUI_Index"] = df_PUI_mapping.index
            df_PUI_mapping["PART_12NC"] = df_PUI_mapping["PART_12NC"].astype(str)
            df_PUI_mapping["COMP_12NC"] = df_PUI_mapping["COMP_12NC"].astype(str)
            df_PUI_mapping.insert(0, 'PUI_Index', df_PUI_mapping.pop('PUI_Index'))
            df_PUI_mapping.insert(1, 'SC Index', df_PUI_mapping.pop('SC Index'))
            df_PUI_mapping.insert(2, 'BOM_ID', df_PUI_mapping.pop('BOM_ID'))
            df_PUI_mapping.insert(10, 'COMP_12NC', df_PUI_mapping.pop('COMP_12NC'))
            df_PUI_mapping.drop(columns = ["SingleComp_index", "MutiComp", "MutiComp_index"], inplace = True)
            df_PUI_mapping.reset_index(drop= True, inplace = True)

        return df_PUI_mapping, df_component_analyzed_result, self.df_all_valid_supply_path

    def find_all_paths(self, graph, start, end, path=None, unique_paths=None):
        if path is None:
            path = []
        if unique_paths is None:
            unique_paths = set()
        path.append(start)
        if start == end:
            # Convert path to a tuple so it can be added to a set
            unique_paths.add(tuple(path))
        else:
            for node in graph.get(start, []):
                if node not in path:  # Avoid cycles
                    self.find_all_paths(graph, node, end, path.copy(), unique_paths)
        return unique_paths


    def find_unique_path(self, graph, start, end, path=None, unique_paths=None):
        # Find all paths
        all_paths = self.find_all_paths(graph, start, end)
        # Convert each tuple path back to a list if needed
        all_paths = [list(path) for path in all_paths]
        # print("============================================================")
        # print("Feasible Combinations: ")
        # print("({}/{})".format(self.SC_Num, self.input_SC_List_len) , all_paths)
        return all_paths


    def Mtype_Aggregation(self, input_all_paths):
        Info = []
        for x in input_all_paths:
            Info_sub = []
            for y in x:
                Info_sub.append(Mtype[y])
            Info.append(Info_sub)

        #Output
        stacker = []
        for x, y in zip(Info, input_all_paths):
            stacker.append(x)
            stacker.append(y)
        output_df = pd.DataFrame(stacker)
        output_df.columns = [('Element_' + str(x + 1)) for x in range(len(output_df.columns))]
        output_df["SC Index"] = self.SC_Num
        output_df.insert(0, 'SC Index', output_df.pop('SC Index'))
        return output_df

    def SC_Structure_Extraction(self, input_PUI_ref, PUI_valid_path_list, consumed_plant, BOM_id):
        SC_Stack = []
        idx = 0
        
        for x in PUI_valid_path_list:
            idx +=1
            for y in range(len(x)):
                try:
                    SC_Stack.append([idx, x[y], x[y+1], input_PUI_ref.loc[(input_PUI_ref["PART_12NC"] == x[y]) & (input_PUI_ref["COMP_12NC"] == x[y+1]), "PLANT"].unique()])
                except:
                    # SC_Stack.append(["", "", "", ""])
                    continue

            
        df_SC_analyezd_Stack = pd.DataFrame(SC_Stack, columns = ["Rolling Result - Combination", "Produced", "Consumed", "Site"])
        #Check if target plant is in Available Plant for each combination
        df_SC_analyezd_Stack["Target Plant"] = consumed_plant
        df_SC_analyezd_Stack["BOM_ID"] = BOM_id
        df_SC_analyezd_Stack["Available Plant"] = df_SC_analyezd_Stack.apply(lambda x: True if consumed_plant in x["Site"] else False, axis = 1)


        for c in df_SC_analyezd_Stack["Rolling Result - Combination"].unique():
            if c != '':
                each_comb = df_SC_analyezd_Stack.loc[(df_SC_analyezd_Stack["Rolling Result - Combination"] == c)]
                if (each_comb["Available Plant"] == False).any() == True:
                    df_SC_analyezd_Stack.loc[(df_SC_analyezd_Stack["Rolling Result - Combination"] == c), "Valid_Plant"] = False
                else:
                    df_SC_analyezd_Stack.loc[(df_SC_analyezd_Stack["Rolling Result - Combination"] == c), "Valid_Plant"] = True

        df_SC_analyezd_Stack.fillna("", inplace = True)
        return df_SC_analyezd_Stack

    def MultiComponent_Checking(self, input_PUI_anlyzied_slice_stack, input_SC_dataframe):
        if len(input_PUI_anlyzied_slice_stack) == 0:
            return pd.DataFrame([])
        else: 
            for bom_id in tqdm(input_PUI_anlyzied_slice_stack["BOM_ID"].unique(), total = len(input_PUI_anlyzied_slice_stack["BOM_ID"].unique())):
                df_extract_from_bom_id = input_PUI_anlyzied_slice_stack.loc[(input_PUI_anlyzied_slice_stack["BOM_ID"] == bom_id)]
                if len(df_extract_from_bom_id["SC Index"].unique()) != len(input_SC_dataframe.loc[input_SC_dataframe["ITEM_BOM_RT_ID"] == bom_id]):
                    input_PUI_anlyzied_slice_stack.loc[(input_PUI_anlyzied_slice_stack["BOM_ID"] == bom_id), "Valid_Comp_Combination"] = False
                    continue
                else:
                    for comb_id in df_extract_from_bom_id["Rolling Result - Combination"].unique():
                        df_extract_from_bom_id_and_comb_id = df_extract_from_bom_id[df_extract_from_bom_id["Rolling Result - Combination"] == comb_id]
                        if (df_extract_from_bom_id_and_comb_id["Valid_Plant"] == False).any():
                            input_PUI_anlyzied_slice_stack.loc[(input_PUI_anlyzied_slice_stack["BOM_ID"] == bom_id) & (input_PUI_anlyzied_slice_stack["Rolling Result - Combination"] == comb_id), "Valid_Comp_Combination"] = False
                        else:
                            input_PUI_anlyzied_slice_stack.loc[(input_PUI_anlyzied_slice_stack["BOM_ID"] == bom_id) & (input_PUI_anlyzied_slice_stack["Rolling Result - Combination"] == comb_id), "Valid_Comp_Combination"] = True
            return input_PUI_anlyzied_slice_stack

    def PUI_result_mapping(self, input_PUI_ref_before_drop_multi_comp, input_SC_analyezd_Stack):
        if len(input_SC_analyezd_Stack) ==0:
            return pd.DataFrame([])
        else:
            for idx, bom_id, p, c, plt, val in tqdm(zip(input_SC_analyezd_Stack["SC Index"], input_SC_analyezd_Stack["BOM_ID"], input_SC_analyezd_Stack["Produced"], input_SC_analyezd_Stack["Consumed"], input_SC_analyezd_Stack["Target Plant"], input_SC_analyezd_Stack["Valid_Comp_Combination"]), total = len(input_SC_analyezd_Stack["SC Index"])):
                if val == True:
                    sub_PUI_index_single = input_PUI_ref_before_drop_multi_comp.loc[(input_PUI_ref_before_drop_multi_comp["PART_12NC"] == p) & (input_PUI_ref_before_drop_multi_comp["COMP_12NC"] == str(c)) & (input_PUI_ref_before_drop_multi_comp["PLANT"] == plt), "SingleComp_index"].dropna().values
                    sub_PUI_index_multi = input_PUI_ref_before_drop_multi_comp.loc[(input_PUI_ref_before_drop_multi_comp["PART_12NC"] == p) & (input_PUI_ref_before_drop_multi_comp["COMP_12NC"] == str(c)) & (input_PUI_ref_before_drop_multi_comp["PLANT"] == plt), "MutiComp_index"].dropna().values
                    sub_PUI_index = set(sub_PUI_index_single).union(set(sub_PUI_index_multi))

                    sub_PUI = input_PUI_ref_before_drop_multi_comp.loc[sub_PUI_index]
                    sub_PUI["SC Index"] = idx
                    sub_PUI["BOM_ID"] = bom_id
                    
                    ###
                    # print("\r({}/{})".format(idx, input_SC_analyezd_Stack["SC Index"].max), bom_id, p, c, plt, val, "sub_PUI_index: {}".format(sub_PUI_index), end = "", flush = True)

                    if len(sub_PUI_index) != 0:
                        self.PUI_stack = pd.concat([self.PUI_stack, sub_PUI])

            return self.PUI_stack


    


### Result Checking and Exporting

In [9]:
# Reuslt_PUI_anlyzied_slice.insert(0, 'SC Index', Reuslt_PUI_anlyzied_slice.pop('SC Index'))
# Reuslt_PUI_anlyzied_slice

# Result_SP.insert(0, 'SC Index', Result_SP.pop('SC Index'))
# Result_SP

In [45]:
from styleframe import StyleFrame, Styler, utils

class General_function:
     def __init__(self):
          self.today = (datetime.datetime.today()).strftime('%Y%m%d')
          self.output_path = "./Output/Result_table_{}.xlsx".format(self.today)

     def Export_to_excel_with_style(self, input_Result, input_Result_SP, input_Reuslt_PUI_anlyzied_slice):
          Result_frame = General_function().style_changes(input_Result, 
                                                          [["PUI_Index", "SC Index", "BOM_ID"], ["PART_12NC", "TYPE", "COMP_12NC"], ["COMP_12NC_LIST"], ["PLANT"]],
                                                          ["yellow", "#85C1E9", "#85C1E9", "green"],
                                                          )
          Result_drop_dup_frame = General_function().style_changes(input_Result.drop(columns=["SC Index", "BOM_ID"]).drop_duplicates().sort_values(by = "PUI_Index"), 
                                                          [["PUI_Index"], ["PART_12NC", "TYPE", "COMP_12NC"], ["COMP_12NC_LIST"], ["PLANT"]],
                                                          ["yellow", "#85C1E9", "#85C1E9", "green"],
                                                          )
          Result_SP_frame = General_function().style_changes(input_Result_SP.reset_index(drop = True), 
                                                          [["SC Index", "Rolling Result - Combination"], ["Produced", "Consumed"], ["Target Plant"], ["BOM_ID"]],
                                                          ["yellow", "#85C1E9", "green"],
                                                          )

          Reuslt_PUI_anlyzied_slice_frame = General_function().style_changes(input_Reuslt_PUI_anlyzied_slice.reset_index(drop = True),
                                                          [["SC Index"]],
                                                          ["yellow"],
                                                          )
          #Export to excel
          excel_writer = StyleFrame.ExcelWriter(self.output_path)
          Result_frame.to_excel(excel_writer, sheet_name = "Result - PUI Mapping", index= False, header = True)
          Result_drop_dup_frame.to_excel(excel_writer, sheet_name = "Result - PUI Mapping (Drop_dup)", index= False, header = True)
          Result_SP_frame.to_excel(excel_writer, sheet_name = "Analyzed - Rolling Supply Path", index= False, header = True)
          Reuslt_PUI_anlyzied_slice_frame.to_excel(excel_writer, sheet_name = "Raw - Rolling Supply Chain", index= False, header = True)
          excel_writer.save()

          return "Data exported completedly !"
          
     def style_changes(self, input_frame, update_columns_combination, color_combination):
          #Summary Table
          if len(input_frame) == 0:
               return StyleFrame(pd.DataFrame([]))
          else:
               sf_output_ref = StyleFrame(input_frame)
               for h_col in input_frame.columns:
                    sf_output_ref[h_col] = input_frame[h_col].fillna("").values
                    if h_col not in ["EFF_FR_DATE", "EFF_TO_DATE"]:
                         sf_output_ref[h_col] = sf_output_ref[h_col].astype(str)
                    header_width = input_frame[h_col].astype(str).str.len().max() + 5
                    if header_width <= len(h_col):
                         header_width = len(h_col) + 5
                    sf_output_ref.apply_column_style(cols_to_style = h_col,
                                                  styler_obj=Styler(font=utils.fonts.calibri, font_size= 11),
                                                  width=header_width,
                                                  style_header=False
                                                  )

                    
               sf_output_ref.apply_headers_style(styler_obj = Styler(font = 'Calibri', font_size = 11, bg_color = "#FDEBD0"), cols_to_style = input_frame.columns)
               for col_list, color in zip(update_columns_combination, color_combination):
                    sf_output_ref.apply_headers_style(styler_obj = Styler(font = 'Calibri', font_size = 11, bg_color = color), cols_to_style = col_list)

               return sf_output_ref     


### Execution 

In [47]:

if __name__=='__main__':
    print("Data loaded in {} mins.".format(round((time.time() - Program_Start_Time)/60, 2)))
    Result, Result_SP, Reuslt_PUI_anlyzied_slice = PUI_Structure_Rolling().main(df_JDA_input, df_agg_final, df_agg)

    # Export to excel with style
    print("\n>> Result is now exporting to excel files ...... ")
    General_function().Export_to_excel_with_style(Result, Result_SP, Reuslt_PUI_anlyzied_slice)
    print(">> Export Process is now completed !")

    os.startfile(os.getcwd().replace("\\", "/") + "/Output/")



Data loaded in 24.46 mins.
Rolling Supply Path ===========================================================


100%|██████████| 23/23 [00:00<00:00, 122.79it/s]



Multi-Component Checking ======================================================


100%|██████████| 13/13 [00:00<00:00, 450.90it/s]



PUI Mapping Result ============================================================


100%|██████████| 83/83 [00:01<00:00, 80.06it/s] 



>> Result is now exporting to excel files ...... 
>> Export Process is now completed !


### Completion Message

In [12]:
print("Analysis Process End ! Time spent: {time_spent} mins".format(time_spent = round((time.time()-Program_Start_Time)/60, 2)))



Analysis Process End ! Time spent: 0.61 mins
